# Reto Dotar de “habilidades” a los monos

Los modos podrán invertir en un universo de 24.000 fondos de inversión (tendréis 5 años de histórico). 

La fecha de la inversión inicial ya no será aleatoria. Todos invertirán, por primera vez, el primer día del segundo año de datos disponibles (para que los monos dispongan de un histórico de datos, de máximo un año, para hacer los cálculos).

A cada modo se le asignará al "nacer", de manera aleatoria, una de las siguientes formas de invertir:

 - Inversión aleatoria en N activos (deberá ser variable con cada mono)
 - Markowitz en N activos (deberá ser variable con cada mono). Usarán 1 año de histórico para hacer los cálculos.
 - Sharpe: invertirá en N fondos (deberá ser variable con cada mono), que tengan un ratio de Sharpe mayor a un umbral de 0. La ventana de datos con la que hacer el cálculo será de 90 días para cada mono.
 - Alpha de Jensen: invertirá en N fondos (deberá ser variable con cada mono) que tengan Alpha positiva en relación con el MSCI. La ventana de datos con la que hacer el cálculo será de 120 días para cada mono.
 
Cada mono calculará el rendimiento obtenido cada 30 días, rebalanceará la carterá y mantendrá su estilo de inversión.
 
__Se valorará__

 - La consecución del objetivo solicitado (añadir las habilidades): debéis mostrar una tabla con la distribución de las rentabilidades obtenidas por habilidad y en global.
 - El número de simulaciones realizadas: se debe alcanzar los 100 millones de monos.
 - El tiempo de ejecución: debéis conocer el tiempo de ejecución (es decir, debe ser ejecutable, aunque dure varias horas). Más de 24h no es admisible.
 - Argumentación del sistema de generación de carteras para cada una de las habilidades (selección de activos y asignación de recursos). 

__Preguntas que debes hacerte, resolver y argumentar con detalle en el inicio del script:__

 - Los activos no tienen los mismos días cotizados, ¿cómo los homogeneizamos?
 - Hay días en los que la mayoría de los activos no tienen cotización ¿qué hacemos?
 - ¿Con cuántos activos nos quedamos y por qué?
 - Para cada una de las habilidades: 
     - ¿Cuántos activos seleccionamos y cómo los seleccionamos? Por ejemplo, para un mono que invierte en función de Sharpe, ¿cómo construyes la cartera?, ¿conoces el universo de activos elegibles en cada instante de tiempo? Argumenta, con detalle, cómo construyes la cartera para cada una de las habilidades.
     - ¿Cómo asignamos peso a los activos seleccionados?
 - ¿Qué hacemos cuando no se puede construir una cartera? Por ejemplo, si no hubiese ningún activo con ratio de Sharpe positivo, o ningún activo estuviera generando Alpha por encima del MSCI. ¿Cómo resuelves este problema?





In [1]:
import numpy as np
import pandas as pd
import random
import time
import sys 
from datetime import timedelta
import cvxpy as cp

## Cargamos, y preparamos, los datos con los que vamos a trabajar

__Leemos los navs del pickle (diccionario)__

In [3]:
navs = pd.read_pickle('navs.pickle')

In [4]:
keys = list(navs.keys())
print(len(keys))
navs[keys[0]]

24822


,isin,allfunds_id,nav,name
date,,,,
2016-01-05,LU0171310443,90,16.47,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
2016-01-06,LU0171310443,90,16.19,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
2016-01-07,LU0171310443,90,15.68,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
2016-01-08,LU0171310443,90,15.59,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
2016-01-11,LU0171310443,90,15.26,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
...,...,...,...,...
2021-07-12,LU0171310443,90,70.55,"BGF WORLD TECHNOLOGY ""A2"" (EUR) (FR)"
2021-07-13,LU0171310443,90,70.85,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"
2021-07-14,LU0171310443,90,71.19,"BGF WORLD TECHNOLOGY ""A2"" (EUR)"


La serie ya tiene quitados los sábados y domingos

Comprobamos si los navs precisan homogeneización

In [5]:
for key in navs.keys():
    
    if navs[key].shape[0] != navs[keys[0]].shape[0]:
        
        print("Es necesario homogeneizar")
        break

Es necesario homogeneizar


Extraemos las fechas iniciales y finales para realizar la homogeneización

In [6]:
fecha_inicial = navs[keys[0]].index[0]
fecha_final = navs[keys[0]].index[-1]

for key in navs.keys():

    if navs[key].index[0] < fecha_inicial:
    
        fecha_inicial = navs[key].index[0]
        
    if navs[key].index[-1] > fecha_final:
        
        fecha_final = navs[key].index[-1]
        
print(fecha_inicial)
print(fecha_final)

2016-01-05 00:00:00
2021-07-16 00:00:00


__Cargamos el MSCI__

In [7]:
msci= pd.read_csv("MSCI.csv", index_col="Fecha",
                  usecols=["Fecha", "Último"],
                  parse_dates=True,
                  infer_datetime_format= "%Y-%m-%d",
                  thousands=".", decimal=",")

msci.rename(columns={'Último': 'MSCI'}, inplace=True)
msci.sort_index(inplace=True)

print(msci.shape)
msci

(1444, 1)


,MSCI
Fecha,
2016-01-05,1630.64
2016-01-06,1610.17
2016-01-07,1576.32
2016-01-08,1561.47
2016-01-11,1557.59
...,...
2021-07-12,3068.27
2021-07-13,3059.28
2021-07-14,3058.59


Comprobamos que el DF no tiene ningún NA

In [8]:
msci.isnull().any()

MSCI    False
dtype: bool

__Cargamos el maestro de valores__

In [9]:
maestro = pd.read_csv('maestro.csv')

In [10]:
maestro.head()

,Unnamed: 0,isin,allfunds_id,asset,asset_type,class_code,clean_share,currency,geo_area,geo_zone,inception_at,income,management_fee,manager_id,manager_name,name,ongoing_charges
0,20,IE00B70DMB28,18152,7.0,ALS,A,False,EUR,3.0,JP,2017-10-11,False,2.00,3,NOMURA INVESTMENT SOLUTIONS PLC,"NOMURA ALPHA JPN LONG SHORT ""A"" (EURH) A",2.42
1,21,LU0331286657,206399,4.0,RVG,C2,False,EUR,3.0,ASP,2003-04-21,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""C2"" (EUR)",3.16
2,22,LU0252964605,27584,4.0,RVG,D2,True,EUR,3.0,ASP,2006-05-19,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""D2"" (EUR)",1.17
3,23,LU1495983089,12993,4.0,RVG,I2,False,USD,3.0,ASP,2016-09-28,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""I2"" (USD)",0.89
4,24,LU0171290587,8123,4.0,RVG,E2,False,EUR,3.0,ASP,2003-04-25,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""E2"" (EUR)",2.41


 - asset: Tipo de activo al que pertenece el fondo del 0 al 7 (mide el riesgo)
 - asset_type: Abreviatura del tipo de activo. Se pueden construir familias con esta característica.
 - class_code: Clase del fondo. Letra con la que se identifica.
 - clean_share: No tiene comisión de retrocesión. Es decir, el fondo no paga comisión al distribuidor.
 - geo_area: Área geográfica a la que pertenece el fondo (1: 'america', 2: 'europa', 3: 'asia', 4: 'global')
 - geo_zone: Zona geográfica dentro del área a la que pertenece el fondo.
 - inception_at: Fecha de creación del fondo
 - income: Indicador política de distribución de dividendos. Si el fondo es de distribución o acumulación (si reparte dividendos o no)
 - management_fee: Comisión Gestión (%)
 - ongoing_charges: Total de gastos del fondo según aparece en el KIID del fondo (management_fee + custodia y administración)

Extraemos los identificadores del maestro de valores (allfunds_id)

In [11]:
identificadores = maestro.loc[:,'allfunds_id']
identificadores.shape

(24903,)

El número de fondos no coincide en el pickle de navs y el maestro. Buscamos las diferencias

In [12]:
list_difference = [item for item in identificadores if item not in keys]
len(list_difference)

81

__Generamos un DF con el que trabajar__

Ponemos los fondos que tenemos en (columnas) y fechas (filas)

In [13]:
fechas = pd.date_range(start=fecha_inicial,end=fecha_final, freq='B')
df_fondos = pd.DataFrame(np.zeros((fechas.shape[0], identificadores.shape[0])),index=fechas,columns=identificadores)

for funds_id in navs.keys():
    df_fondos.loc[:,funds_id] = navs[funds_id].nav

Nos quedamos con aquellos fondos que tengan más del 60% de los datos rellenos

In [14]:
print(df_fondos.shape)
df_fondos = df_fondos.loc[:,df_fondos.isna().sum(axis=0)<df_fondos.shape[0]*0.4]
print(df_fondos.shape)

(1444, 24903)
(1444, 20286)


Rellenamos los datos faltantes

In [15]:
df_fondos = df_fondos.fillna(method = 'ffill')
df_fondos = df_fondos.fillna(method = 'bfill')

Quitamos aquellos fondos que tienen nav 0 en todas sus fechas: los fondos que estaban en el maestro de valores, pero que no tenían Nav

In [16]:
df_fondos=df_fondos.drop(list_difference, axis=1)
df_fondos.shape

(1444, 20205)

In [17]:
df_fondos

allfunds_id,206399,27584,12993,8123,11125,8109,11108,27583,10013,157417,...,230294,19023,42535,267220,34808,34806,249846,2913,2914,255530
2016-01-05,23.46,30.93,10.00,26.92,28.92,28.77,30.90,33.22,21.07,10.30,...,100.35,99.97,100.00,94.73,99.95,99.95,127.43,100.23894,100.23600,1390.45
2016-01-06,23.22,30.61,10.00,26.64,28.66,28.47,30.63,32.92,20.92,10.34,...,100.33,99.97,100.00,93.86,99.95,99.95,126.86,100.23894,100.23600,1391.02
2016-01-07,22.42,29.57,10.00,25.74,27.92,27.51,29.84,32.08,20.45,10.34,...,100.25,99.97,100.00,92.57,99.95,99.95,123.59,100.23894,100.23600,1392.28
2016-01-08,22.28,29.37,10.00,25.57,27.80,27.32,29.71,31.94,20.38,10.32,...,100.26,99.97,100.00,92.39,99.95,99.95,119.88,100.23894,100.23600,1393.15
2016-01-11,22.03,29.05,10.00,25.28,27.55,27.02,29.44,31.66,20.20,10.33,...,100.20,99.97,100.00,91.60,99.95,99.95,117.92,100.23894,100.23600,1393.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-12,29.10,42.81,14.39,34.80,41.50,38.22,45.58,51.05,32.76,11.11,...,92.92,103.91,96.47,107.52,120.90,122.33,228.05,109.47140,120.46439,1524.30
2021-07-13,29.10,42.81,14.39,34.80,41.50,38.22,45.58,51.05,32.76,11.11,...,92.93,103.92,96.47,107.81,120.87,122.30,228.05,109.47140,120.46439,1524.30
2021-07-14,29.10,42.81,14.39,34.80,41.50,38.22,45.58,51.05,32.76,11.10,...,93.01,103.93,96.48,107.97,120.97,122.40,228.05,109.47140,120.46439,1571.50
2021-07-15,29.10,42.81,14.39,34.80,41.50,38.22,45.58,51.05,32.76,11.10,...,92.92,103.94,96.49,107.60,121.21,122.64,228.05,109.47140,120.46439,1573.14


Comprobamos que el índice está ordenado

In [18]:
all(df_fondos.sort_index().index == df_fondos.index)

True

Ambos DF (msci y fondos), tienen el mismo tamaño. Nos queda confirmar que tenemos los mismos días

In [19]:
all(df_fondos.index == msci.index)

True

## Habilidades de los monos

__Alpha de Jensen__

α=Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))

β= covarianza (Rentabilidad_cartera, Rentabilidad_mercado) / varianza (Rentabilidad_mercado)

Alpha de Jensen: invertirá en N fondos (variable) que tengan Alpha positiva en relación con el MSCI. 

La ventana de datos con la que hacer el cálculo será aleatoria de 120 días para cada mono.

La función del Alpha calculará el universo de fondos que cumplen la condición de Alpha positivo en relación al MSCI, en un momento dado. Una vez tengamos el universo, la inversión será aleatoria en dicho universo.

En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

In [20]:
universe = df_fondos.iloc[0:120,]
ind_msci = msci.iloc[0:120]

# Generamos las rentabilidades logarítmicas para el índice, la rentabilidad libre de riesgo y los activos.
rent_activos = np.log(universe).diff().dropna()
rent_msci = np.log(ind_msci).diff().dropna()
rent_libre_riesgo = 0

# Calculamos la varianza, covarianza y beta.
varianza_msci = rent_msci.var()

El cálculo de la covarianza no parece trivial. Vamos a analizarlo

En primer lugar, tenemos que transponer para hacer los cálculos, ya que numpy calcula por filas y R por columnas

In [21]:
np.cov(rent_activos.iloc[:,0].T, rent_msci.T)

array([[1.55544546e-04, 6.11625480e-05],
       [6.11625480e-05, 7.54996452e-05]])

Esto devuelve 

- cov(a,a) cov(a,b)
- cov(a,b) cov(b,b)

Por lo que si quiero obtener la cov (a,b), tengo que hacer lo siguiente:

In [22]:
np.cov(rent_activos.iloc[:,0].T, rent_msci.T)[0][1]

6.11625479622914e-05

Si quiero extrapolar el cálculo a todos los activos, tengo varias maneras de hacerlo

- cov_act_msci = np.cov(rent_activos.iloc[:,0:-1].T, rent_msci.T) --> Da una matriz de (20205, 20205), la última columna tiene los resultados que queremos.
- cov_act_msci = np.cov(rent_activos.T, rent_msci.T) --> Da una matriz de (20206, 20206), la última columna es el MSCI. Esta columna es el resultado que queremos. 
- cov_act_msci = pd.concat([rent_activos,rent_msci], axis=1).cov() --> Da una matriz de (20206, 20206), la última columna es el MSCI. Esta columna es el resultado que queremos.
- Hacer un bucle act a act, y meter el resultado en una lista --> No vamos a hacerlo así

In [23]:
def universe_alpha(universe, ind_msci):

    '''
    Alpha de Jensen: invertirá en N fondos (variable) que tengan Alpha positiva en relación con el MSCI. 
    La ventana de datos con la que hacer el cálculo será de 120 días para cada mono.

    La función del Alpha calculará el universo de fondos que cumplen la condición de Alpha positivo en relación al MSCI, en un momento dado. 
    Una vez tengamos el universo, la inversión será aleatoria en dicho universo.
    En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

    La función recibe los DF de fondos y msci, y devuelve un índice booleano que modifica el universo de inverión aleatoria.
    α=Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))
    β= covarianza (Rentabilidad_cartera, Rentabilidad_mercado) / varianza (Rentabilidad_mercado)
    '''

    #universe = df_fondos.iloc[0:120,]
    #ind_msci = msci.iloc[0:120]

    # Generamos las rentabilidades logarítmicas para el índice, la rentabilidad libre de riesgo y los activos.
    rent_activos = np.log(universe).diff().dropna()
    rent_msci = np.log(ind_msci).diff().dropna()
    rent_libre_riesgo = 0

    # Calculamos la varianza
    varianza_msci = rent_msci.var()

    # Calculamos la covarianza
    cov_act_msci = pd.concat([rent_activos,rent_msci], axis=1).cov()
    cov_act_msci = cov_act_msci.iloc[0:-1,-1] 

    # Calculamos la Beta y el Alpha
    if varianza_msci[0] == 0:
        
        # beta = 1
        alpha = rent_activos.iloc[-1,:] - rent_msci.iloc[-1,0]
        
    else:
        
        beta = cov_act_msci/varianza_msci[0]
        
        # Calculamos el Alpha para cada activo: Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))
        alpha = rent_activos.iloc[-1,:] - beta.iloc[:] * rent_msci.iloc[-1,0]

    # Devolvemos el universo de activos elegibles: aquellos que tienen alpha > 0
    # Si ningún activo cumpliera el criterio, el universo de activos elegibles sería el inicial (todos los activos).
    if sum(alpha > 0)==0:
        universo_elegible_alpha = alpha < 0
    else:
        universo_elegible_alpha = alpha > 0

    return universo_elegible_alpha

__Ratio de Sharpe__

Ratio de Sharpe= (Rent activo- Rentabilidad libre de riesgo)/ desviación típica activo

Sharpe: invertirá en N fondos (variable), que tengan un ratio de Sharpe mayor a un umbral de 0,5. La ventana de datos con la que hacer el cálculo será aleatoria entre 90 días para cada mono.

La función de Sharpe calculará el universo de fondos que cumplen la condición de Sharpe positivo, en un momento dado. Una vez tengamos el universo, la inversión será aleatoria en dicho universo.

En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

In [42]:
def universe_sharpe(universe, df_fondos, umbral=0.5):

    '''
    Sharpe: invertirá en N fondos (variable), que tengan un ratio de Sharpe mayor a un umbral de 0,5. 
    La ventana de datos con la que hacer el cálculo será de 90 días para cada mono.

    La función calculará el universo de fondos que cumplen la condición de Sharpe > umbral, en un momento dado. 
    Una vez tengamos el universo, la inversión será aleatoria en dicho universo.
    En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

    La función recibe los DF de fondos y un umbral, y devuelve un índice booleano que modifica el universo de inverión aleatoria.
    Ratio de Sharpe= (Rent activo- Rentabilidad libre de riesgo)/ desviación típica activo
    '''
    
    #umbral = 0.5
    #universe = df_fondos.iloc[0:90,]

    # Generamos las rentabilidades para los activos y calculamos la desviación típica
    rent_activos = np.log(universe).diff().dropna()
    rent_libre_riesgo = 0
    desv_act = df_fondos.std(axis=0)

    # Calculamos el ratio de Sharpe
    sharpe = rent_activos.iloc[-1,:]/desv_act

    # Las desviaciones que eran = 0 se han convertido en NA. Lo solventamos
    sharpe[desv_act == 0] = 0

    # Devolvemos el universo de activos elegibles: aquellos que tienen Sharpe > 0.5
    # Si ningún activo cumpliera el criterio, el universo de activos elegibles sería el inicial (todos los activos).
    if sum(sharpe > umbral)==0:
        universo_elegible_sharpe = sharpe < umbral
    else:
        universo_elegible_sharpe = sharpe > umbral

    return universo_elegible_sharpe

__Inversión aleatoria__

Inversión aleatoria en N activos (deberá ser variable con cada mono)

La función recibirá el universo de activos y devolverá la cartera: Nº de activos, qué activos y qué peso (las funciones Alpha y Sharpe pasarán un sub-universo para hacer los cálculos)

In [25]:
def inversion_aleatoria(universe):

    '''
    El cómo asignamos pesos a los activos es la primera decisión importante a tomar. El problema es el siguiente:
    ¿Cómo hago que todos los fondos seleccionados reciban un peso y, a la vez, permito que uno o dos activos se puedan llevar un % muy alto o, incluso, el 100%?
    Pensemos en las posibilidades existentes:

        1) Asignamos un peso aleatorio a cada activo y reescalamos los pesos a 1. Bueno: todos los activos seleccionados tendrán peso. Malo: ningún activo tendrá un % alto en carteras de más de 3 activos.
        2) Recorremos aleatoriamente los activos seleccionados, y les asignamos el % que salga. Al llegar a 100% paramos. Bueno: Los activos pueden tener pesos muy altos en la cartera. Malo: muchos activos se quedarán sin peso.

    La segunda solución parece la mejor. Pero tras estudiarla asigna peso a 3 - 5 activos como máximo en el 99% de los casos. Por lo que carteras de hasta 30 fondos no van a generarse.

    Si queremos poder construir carteras entre 5 y 30 activos, tendríamos que utilizar la primera opción. Que estén distribuidos de manera que ningún fondo tenga un porcentaje muy alto, diversifica el riesgo. Aceptable en este caso.
    '''

    # universe_size = universe.shape[1] # Cuando es un DF
    universe_size = universe.size # Cuando es una serie
    numero_fondos = random.randint(5,30)
    fondos_elegidos = np.random.choice(universe_size, numero_fondos, replace=False) # posiciones de los fondos
    # id_fondos_elegidos = universe.columns[fondos_elegidos] # Cuando es un DF
    id_fondos_elegidos = universe.index[fondos_elegidos] # Cuando es una serie

    pesos_asignados = np.random.random(numero_fondos)
    pesos_asignados = pesos_asignados/sum(pesos_asignados)
    
    return id_fondos_elegidos, pesos_asignados, fondos_elegidos

__Markowitz__

Markowitz en N activos (deberá ser variable con cada mono). Usarán 1 año de histórico para hacer los cálculos.

La función sacará un nº aleatorio de activos y devolverá la cartera más eficiente después de N iteraciones.

In [26]:
def generador_pesos(sub_universo, simulaciones = 10000):
    
    '''
    Recibe un sub universo y devuelve una matriz de pesos donde las filas son el nº de simulaciones y las columnas los activos del sub universo
    Cada fila (simulación), tiene un número distinto de activos que reciben peso
    No todos los activos, en todas las simulaciones pueden recibir asignación de capital (peso)
    La solución es crear un conjunto de simulaciones, dentro del sub universo, para poder calcular cúal es la más eficiente
    EL objetivo de esta función es devolver una matriz de pesos, donde cada simulación tiene una asignación de pesos distinta.
    '''
    
    num_activos = sub_universo.shape[1]
    aleatorios = np.random.random((simulaciones,num_activos))

    umbral = 0.8
    aleatorios[aleatorios<umbral] = 0

    # Buscamos y eliminamos las simulaciones que no tienen ningún activo seleccionado
    seleccion = ~(aleatorios.sum(axis=1) == 0)
    aleatorios = aleatorios[seleccion,:]

    # Asignamos los pesos a cada cartera
    # Para poder usar los aleatorios, dado que todos los activos seleccionados superan un umbral, debo restar el umbral
    # Si reescalase el peso, ningún activo de una cartera de 3 elementos superaría el 50% - 60% de peso. No puedo asignar los pesos reescalando.
    aleatorios[aleatorios!=0] = (aleatorios[aleatorios!=0] - umbral) * random.randint(1,15)

    for sim in range(aleatorios.shape[0]):

        vector_pesos = aleatorios[sim,:]    
        posiciones = list(set(np.where(vector_pesos!=0)[0]))

        # Obtenemos el órden en el que vamos a asignar los pesos
        # posiciones_aleatorias = random.sample(posiciones, len(posiciones)) # Deprecado
        random.shuffle(posiciones)

        peso = 0
        for pos in posiciones:
            if peso < 1:
                peso += aleatorios[sim, pos]

                if peso > 1:            
                    exceso = peso - 1
                    aleatorios[sim, pos] = aleatorios[sim, pos] - exceso
                    peso = 1            

            elif peso == 1:
                aleatorios[sim, pos] = 0
    
    # Las carteras no van a sumar siempre un 100% invertido
    # Dependerá del umbral y del número por el que multipliquemos los aleatorios, una vez restado el umbral
        
    return aleatorios

In [28]:
def frontera_markowitz(universe ,simulaciones=10000):

    '''
    Si calculásemos la frontera de Markowitz con todos los activos, un número suficientemente grande de simulaciones
    todos los monos que usasen este método de inversión, tendrían "la misma cartera"
    Para evitarlo, la función trabaja con un sub universo de inversión diferente para cada mono
    con un número de inversiones aleatorias no excesivamente grande, para que exista diversidad, 
    tengamos un tiempo de ejecución razonable.
    '''
  
    # Elegimos aleatoriamente el número de activos con los que trabajar, y cuales serán (sub universo)
    universe_size = universe.shape[1]
    numero_fondos = random.randint(5,30)
    fondos_elegidos = np.random.choice(universe_size, numero_fondos, replace=False) # posiciones de los fondos
    id_fondos_elegidos = universe.columns[fondos_elegidos]
    sub_universo = universe.iloc[:,fondos_elegidos]

    # Calculamos la rentabilidad de los activos
    rent_activos = np.log(sub_universo).diff().dropna()

    # Calculamos la matriz de varianzas / covarianzas
    matriz_covarianzas = rent_activos.cov()
    matriz_covarianzas = matriz_covarianzas.to_numpy(dtype='float') 

    # Calculamos la rentabilidad diaria del periodo para cada activo: LN(precio final/ precio inicial)/nº de datos
    precio_inicial = sub_universo.iloc[0,:]
    precio_final = sub_universo.iloc[-1,:]
    rentabilidad_diaria = np.log(precio_final / precio_inicial) / sub_universo.shape[0]

    # Sacamos la matriz de pesos para cada cartera
    matriz_pesos = generador_pesos(sub_universo, simulaciones = simulaciones)

    # Calculamos la rentabilidad de la cartera en función de los pesos 
    auxiliar = rentabilidad_diaria.values * matriz_pesos
    rentabilidad_carteras = auxiliar.sum(axis=1)

    # Calculamos el riesgo de la cartera (desviación), en función de los pesos
    auxiliar = np.dot(matriz_pesos, matriz_covarianzas) # Reutilizo la matriz auxiliar con otro propósito para no sobrecargar la RAM
    riesgo_carteras = pow((auxiliar * matriz_pesos).sum(axis=1), 0.5)

    # Calculamos la eficiencia de la cartera (pendiente), en función de los pesos
    # Existen posiciones que van a dar error, 0/0. Las localizo y hago que 0/1 = 0
    posiciones_0entre0 = (rentabilidad_carteras==0) * (riesgo_carteras==0)
    riesgo_carteras[posiciones_0entre0] = 0.0000001
    eficiencia_carteras = rentabilidad_carteras / riesgo_carteras

    # Localizamos la cartera con mayor eficiencia
    max_eficiencia = max(eficiencia_carteras)

    # Localizamos la composición de la cartera eficiente en el sub universo  
    posicion_cartera_eficiente = np.where(eficiencia_carteras == max_eficiencia)[0][0]
    cartera_eficiente = matriz_pesos[posicion_cartera_eficiente,:]    
    posicion_activos_eficientes = np.where(cartera_eficiente!=0)

    peso_activos_eficientes = cartera_eficiente[posicion_activos_eficientes]            
    isin_activos_eficientes = rent_activos.columns[posicion_activos_eficientes[0]] 

    # Situamos la cartera eficiente en el universo global
    id_fondos_elegidos = isin_activos_eficientes
    pesos_asignados = peso_activos_eficientes
    fondos_elegidos = fondos_elegidos[posicion_activos_eficientes[0]] # Posición de los fondos en el universo completo            

    return id_fondos_elegidos, pesos_asignados, fondos_elegidos

Hacemos una prueba para comprobar que el tiempo de ejecución es asumible para hacerlo con cada mono

In [29]:
start_time = time.time()

universe_prueba = df_fondos.iloc[0:240,]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = frontera_markowitz(universe_prueba, simulaciones = 10000)

print(id_fondos_elegidos)
print(pesos_asignados)
print(posiciones_fondos_elegidos)

print("--- %s seconds ---" % (time.time() - start_time))

Int64Index([34801, 255748, 136693], dtype='int64', name='allfunds_id')
[0.47079518 0.49654367 0.03266115]
[12603 14717  2194]
--- 0.11066317558288574 seconds ---


## Análisis de tiempo

Si empezamos los cálculos el primer día del 2º año, y tenemos que hacer los cálculos cada 30 días

¿Cuantas veces tendremos que calcularlo todo?

El año 2016 lo usaremos únicamente como histórico, por lo que empezaremos el primer día bursátil del 2017

In [30]:
veces = pd.date_range('2017-01-01', df_fondos.index[-1], freq='BMS').shape[0]
f'{veces} veces'

'55 veces'

#### Inversión basada en el Alpha de Jensen

Invocamos al cálculo del universo de alpha para que limite el universo de activos elegibles.

Limitar el universo de activos se una vez, cada 30 días.

¿Cuantas veces tendremos que hacer el cálculo?

In [31]:
start_time = time.time()

universe_prueba = df_fondos.iloc[0:120,]
ind_msci = msci.iloc[0:120]

universo_elegible_alpha = universe_alpha(universe_prueba, ind_msci)
universo_elegible_alpha = df_fondos.loc[:,universo_elegible_alpha]

f'Tardará {(time.time() - start_time)* veces} segundos'

'Tardará 178.1594157218933 segundos'

Una vez tengamos delimitado el universo de activos, tendremos que calcular las carteras para 25 millones de monos, N veces

¿Cuanto nos llevará hacer ese cálculo?

In [32]:
start_time = time.time()

universe_prueba = universo_elegible_alpha.iloc[120,:]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria(universe_prueba)

#print(id_fondos_elegidos)
#print(pesos_asignados)
#print(posiciones_fondos_elegidos)

segundos = (time.time() - start_time)* 25_000_000 * veces
horas_alpha = segundos/60/60
f'Tardará {horas_alpha} horas'

'Tardará 763.924585448371 horas'

#### Inversión basada en Sharpe

Invocamos al cálculo del universo de Sharpe para que limite el universo de activos elegibles.

Limitar el universo de activos se una vez, cada 30 días.

In [33]:
start_time = time.time()

universe_prueba = df_fondos.iloc[0:90,]

universe_elegible_sharpe = universe_sharpe(universe_prueba, umbral = 0.5)
universe_elegible_sharpe = df_fondos.loc[:,universe_elegible_sharpe]

f'Tardará {(time.time() - start_time)* veces} segundos'

'Tardará 12.858065366744995 segundos'

Una vez tengamos delimitado el universo de activos, tendremos que calcular las carteras para 25 millones de monos, N veces

In [34]:
start_time = time.time()

universe_prueba = universe_elegible_sharpe.iloc[90,]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria(universe_prueba)

#print(id_fondos_elegidos)
#print(pesos_asignados)
#print(posiciones_fondos_elegidos)

segundos = (time.time() - start_time)* 25_000_000 * veces
horas_sharpe = segundos/60/60
f'Tardará {horas_sharpe} horas'

'Tardará 2307.254407140944 horas'

#### Inversión aleatoria

Invocamos un mono aleatorio que recibe el universo de activos elegibles al completo

Esto se hace para cada mono basado en inversión aleatoria, cada 30 días

Tendremos que calcular las carteras para 25 millones de monos, N veces

In [35]:
start_time = time.time()

universe_prueba = df_fondos.iloc[120,:]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria(universe_prueba)

#print(id_fondos_elegidos)
#print(pesos_asignados)
#print(posiciones_fondos_elegidos)

segundos = (time.time() - start_time)* 25_000_000 * veces
horas_ale = segundos/60/60
f'Tardará {horas_ale} horas'

'Tardará 4954.081442621019 horas'

#### Inversión basada en la frontera de Markowitz

Invocamos un mono que basará su inversión en Markowitz, recibiendo el universo de activos elegibles al completo.

Esto se hace para cada mono, cada 30 días.

Tendremos que calcular las carteras para 25 millones de monos, N veces

In [36]:
# Ojo, no siempre está invertido al 100%

start_time = time.time()

universe_prueba = df_fondos.iloc[0:240,]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = frontera_markowitz(universe_prueba, simulaciones = 10000)

#print(id_fondos_elegidos)
#print(pesos_asignados)
#print(posiciones_fondos_elegidos)
#print(pesos_asignados.sum())

segundos = (time.time() - start_time)* 25_000_000 * veces
horas_mark = segundos/60/60
f'Tardará {horas_mark} horas'

'Tardará 43458.284603224856 horas'

#### Resumen tiempos

El tiempo de limitar los universos de inversión es despreciable

El problema está en calcular 75 millones de aleatorios + 25 millones de Markowitz, N veces

In [37]:
f'Total tiempo = {(horas_alpha + horas_sharpe + horas_ale +horas_mark)/24/365} años'

'Total tiempo = 5.877117013519999 años'

Tenemos que planteárnoslo de otra manera

### Tenemos que pensar de otra manera

Si todas las carteras tuvieran un número fijo de fondos, podríamos sacar todos los aleatorios de golpe y reescalar los pesos (sería la manera más rápida de hacerlo)

El problema es que queremos carteras con números variables de activos

La solución es evidente:

Podríamos sacar un número fijo de activos para cada cartera: 30, por ejemplo

Y calcular una matriz aleatoria booleana para "llevar a cero", aleatoriamente, a varios activos de cada cartera.

De esta manera podemos sacar las carteras 25 millones de monos, de una sola vez.

Y cada cartera tendría activos distintos, con pesos distintos (aunque los activos podrían repetirse).

In [38]:
def inversion_aleatoria_fast(universe, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25):

    '''
    Método de inversión aleatoria capaz de obtener gran cantidad de carteras, de tamaño aleatorio, de una sola vez
    '''
    
    universe_size = universe.size
    
    posiciones_fondos_elegidos = np.random.choice(universe_size, num_fondos_por_cartera * num_carteras, replace=True)
    id_fondos_elegidos = universe.index.values[posiciones_fondos_elegidos]
    pesos_asignados = np.random.random(num_fondos_por_cartera * num_carteras)

    # f'{sys.getsizeof(posiciones_fondos_elegidos)/1_000_000} mb' # Nos da el tamaño en RAM que ocupa el objeto
    posiciones_fondos_elegidos.resize(num_carteras,num_fondos_por_cartera) # Mejor que reshape porque resize lo hace inplace
    id_fondos_elegidos.resize(num_carteras,num_fondos_por_cartera)
    pesos_asignados.resize(num_carteras,num_fondos_por_cartera)

    # Creamos una matriz booleana para anular aleatoriamente los pesos de N fondos de cada cartera (fila)
    anular = np.random.choice(a=[False, True], size=(num_carteras, num_fondos_por_cartera), p=[p, 1-p])  
    pesos_asignados = pesos_asignados * anular

    # Hay que reescalar los pesos por filas (por carteras)
    pesos_asignados = pesos_asignados/pesos_asignados.sum(axis=1, keepdims=True)

    return id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos

Ponemos a prueba cuánto tardaría este nuevo enfoque

El tamaño de cada matriz, si sacamos de golpe 1 millón de carteras, estará entre 120 - 240 MB

Si sacamos de golpe 25 millones, cada matriz ocupará entre 3 y 6 GB

Por lo que repetiremos el proceso 25 veces

In [39]:
start_time = time.time()

universe_prueba = df_fondos.iloc[120,:]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria_fast(universe_prueba, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)

print(id_fondos_elegidos.shape)
print(f'{sys.getsizeof(id_fondos_elegidos)/1_000_000} mb ocupa id_fondos_elegidos')
print(pesos_asignados.shape)
print(f'{sys.getsizeof(pesos_asignados)/1_000_000} mb ocupa pesos_asignados')
print(posiciones_fondos_elegidos.shape)
print(f'{sys.getsizeof(posiciones_fondos_elegidos)/1_000_000} mb ocupa posiciones_fondos_elegidos')

segundos = (time.time() - start_time)* 25 * veces
horas_ale = segundos/60/60
f'Tardará {horas_ale} horas'

(1000000, 30)
240.00012 mb ocupa id_fondos_elegidos
(1000000, 30)
240.00012 mb ocupa pesos_asignados
(1000000, 30)
120.00012 mb ocupa posiciones_fondos_elegidos


'Tardará 0.4458284460835987 horas'

Hemos resuelto el problema para los monos que inviertan basándose en Alpha, Sharpe y de manera aleatoria

Nos queda resolver el problema de los monos que inviertan con Markowitz

Aparentemente, la solución del problema es sencilla. Tenemos una función "generador_pesos", la cual genera, de manera muy eficiente, 10.000 carteras distintas, de la que nos quedamos con la que mayor eficiencia tiene. 

Podríamos incrementar el número de las carteras, y quedarnos con el 10% de las mismas que mejor lo hagan. En vez de solo con la mejor.

El problema de la función "generador_pesos", es que trabaja con un sub universo muy reducido. De 30 activos. Por lo que incrementar únicamente el número de carteras, no solventa el problema.

Pero trabajar con el universo complero hace que el tiempo de ejecución de la función, así como el peso del objeto de devuelve, no sea en absoluto eficiente. 

Parte de la solución pasa por quedarnos con un porcentaje significativo de las carteras más eficientes. Pero habrá que tratar, con mucho cuidado, el tema del universo, peso y tiempo de ejecución.

In [40]:
def frontera_markowitz_fast(universe, num_fondos_por_cartera = 100, num_carteras = 500_000, p = 0.85, porcentaje_util = 0.2):
    '''
    El objetivo es que la función arroje 100.000 carteras (num_carteras * porcentaje_util), si bien no óptimas, sí eficientes.
    El número óptimo de fondos con los que la función trabaja es 100, por tiempo de ejecución y tamaño en RAM
    Eso no significa que cada cartera tenga 100 activos. Cada cartera tendrá un número distinto de activos, gracias al proceso de anulación.
    Se genera una matriz booleana, con una probabilidad de False de p. El objetivo es que las carteras tengan entre 5 y 30 activos.
    De las 500.000 carteras calculadas, la función se queda con el percentil 80, para arriba. 
    '''

    # Sacamos la matriz de pesos para cada cartera
    pesos_asignados = np.random.random(num_fondos_por_cartera * num_carteras)
    pesos_asignados.resize(num_carteras,num_fondos_por_cartera)

    # Creamos una matriz booleana para anular aleatoriamente los pesos de N fondos de cada cartera (fila)
    anular = np.random.choice(a=[False, True], size=(num_carteras, num_fondos_por_cartera), p=[p, 1-p])  
    pesos_asignados = pesos_asignados * anular      
    
    # Quitamos aquellas carteras que no tengan ningún activo con peso asignado
    pesos_asignados = pesos_asignados[anular.sum(axis=1) != 0,:]    

    # Hay que reescalar los pesos por filas (por carteras)
    matriz_pesos = pesos_asignados/pesos_asignados.sum(axis=1, keepdims=True)

    # Elegimos aleatoriamente los activos con los que trabajar (sub universo)    
    universe_size = universe.shape[1]
    posiciones_fondos_elegidos = np.random.choice(universe_size, num_fondos_por_cartera, replace=False)
    id_fondos_elegidos = universe.columns[posiciones_fondos_elegidos]
    sub_universo = universe.iloc[:,posiciones_fondos_elegidos]

    # Calculamos la rentabilidad de los activos
    rent_activos = np.log(sub_universo).diff().dropna()

    # Calculamos la matriz de varianzas / covarianzas
    matriz_covarianzas = rent_activos.cov()
    matriz_covarianzas = matriz_covarianzas.to_numpy(dtype='float') 

    # Calculamos la rentabilidad diaria del periodo para cada activo: LN(precio final/ precio inicial)/nº de datos
    precio_inicial = sub_universo.iloc[0,:]
    precio_final = sub_universo.iloc[-1,:]
    rentabilidad_diaria = np.log(precio_final / precio_inicial) / sub_universo.shape[0]

    # Calculamos la rentabilidad de la cartera en función de los pesos 
    auxiliar = rentabilidad_diaria.values * matriz_pesos
    rentabilidad_carteras = auxiliar.sum(axis=1)

    # Calculamos el riesgo de la cartera (desviación), en función de los pesos
    auxiliar = np.dot(matriz_pesos, matriz_covarianzas) # Reutilizo la matriz auxiliar con otro propósito para no sobrecargar la RAM
    riesgo_carteras = pow((auxiliar * matriz_pesos).sum(axis=1), 0.5)

    # Calculamos la eficiencia de la cartera (pendiente), en función de los pesos
    # Existen posiciones que van a dar error, 0/0. Las localizo y hago que 0/1 = 0
    posiciones_0entre0 = (rentabilidad_carteras==0) * (riesgo_carteras==0)
    riesgo_carteras[posiciones_0entre0] = 0.0000001
    eficiencia_carteras = rentabilidad_carteras / riesgo_carteras

    # Localizamos las carteras con mayor eficiencia
    umbral_eficiencia = np.quantile(eficiencia_carteras, (1-porcentaje_util))    
    pesos_asignados = matriz_pesos[eficiencia_carteras>umbral_eficiencia,:]

    return id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos

In [41]:
start_time = time.time()

universe_prueba = df_fondos.iloc[0:240,]

id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = frontera_markowitz_fast(universe_prueba, num_carteras = 250_000)

print(id_fondos_elegidos.shape)
print(pesos_asignados.shape)
print(posiciones_fondos_elegidos.shape)

segundos = (time.time() - start_time)* 20 * 25 * veces
horas_mark = segundos/60/60
f'Tardará {horas_mark} horas'

(100,)
(50000, 100)
(100,)


'Tardará 7.173154420322843 horas'

Aunque no es tan eficiente como la función "inversion_aleatoria_fast", hemos resuelto el problema del tiempo de ejecución en menos de 24 horas.

¿Qué pasaría si calculásemos la cartera óptima mediante una optimización cuadrática. ¿Obtendríamos un Markowitz "ulta-fast"?

Con la función anterior estamos obteniendo 50_000 carteras, de combinaciones de 100 activos. Pero muchas de esas soluciones están "dominadas" al no encontrarse en la frontera eficiente.

La pregunta es, ¿sería más eficiente trabajar con una optimización cuadrática?

- La idea es que trabajemos con el mismo número de activos. Con 100, por optimización. La optimización cuadrática no puede trabajar con más de 8.000 - 10.000 activos. Aparentemente estamos lejos de ese punto.
- Trabajaríamos con menos carteras por simulación, pero todas estarían en la frontara de eficiencia.
- ¿Cuanto tiempo tardaremos? al tener menos carteras por cada llamada, tendremos que llamar a la función muchas más veces. ¿Vale la pena?
- El objetivo es obtener el mismo output. O lo más parecido posible: 
    - id_fondos_elegidos: 100
    - pesos_asignados: N * 100
    - posiciones_fondos_elegidos: 100

Vamos a investigar si vale la pena

In [291]:
def cuadratic_markowitz(sub_universo, n_samples=1000, gamma_low=-1, gamma_high=10):

    # Calculamos la rentabilidad de los activos
    returns = np.log(sub_universo).diff().dropna()
    
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T*w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  w >= 0]) 
    
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = np.zeros((n_samples, sub_universo.shape[1]))
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()  
        portfolio_weights[i,:] = w.value
        
    return portfolio_weights

In [292]:
start_time = time.time()

num_fondos_por_cartera = 30
n_samples=1000

# Vamos a poner la selección del sub_universo fuera de la función, para no tener que pasar el DF completo en cada llamada
universe_size = df_fondos.shape[1]
posiciones_fondos_elegidos = np.random.choice(universe_size, num_fondos_por_cartera, replace=False)
id_fondos_elegidos = universe.columns[posiciones_fondos_elegidos]
sub_universo = df_fondos.iloc[0:240,posiciones_fondos_elegidos]

portfolio_weights = cuadratic_markowitz(sub_universo = sub_universo, n_samples = 1000)

segundos = (time.time() - start_time)* 25_000_000 / n_samples * veces
horas_mark = segundos/60/60
f'Tardará {horas_mark} horas'

C:\Users\gmele\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 11 times so far.

  warnings.warn(msg, UserWarning)


'Tardará 760.2118700742722 horas'

El tiempo de ejecución no vale la pena.

Para encontrar la cartera óptima, la optimización cuadrática lo hará mejor que la generación de pesos aleatorios (especialmente si los reescalamos). Siempre y cuando trabajemos con un número relativamente bajo de activos. 

Sin embargo, para la generación de 25 millones de monos, con inversión basada en Markowitz, no es la solución que estamos buscando.

Si ejecutamos la función varias veces vemos que, con frecuencia, no converge en una solución: a partir de 50 fondos suele dar error.

Otro problema es que, al estar todas las carteras en la frontera eficiente son, entre sí, muy parecidas en peso. 

## Montamos el programa que generará 100 millones de inversiones aleatorias en menos de 24 horas

¿Cual es la mejor manera de llevar a cabo este proceso?

Tendremos un problema de memoria, dado que tendremos que guardar, cada 30 días, 100 millones de rentabilidades.

- Podríamos ejecutar las 100 simulaciones cada 30 días y guardar la información en una matriz de N filas (repeticiones cada 30 días). Pero probablemente sea la peor manera de hacerlo.
- Podríamos separar las tipologías de monos: 4 estilos de inversión, 25 millones de simulaciones para cada uno. Podríamos calcular los 25 millones e ir avanzando cada 30 días.
- Podríamos generar 1 millón de simulaciones cada vez. Avanzamos cada 30 días y terminamos la simulación por bloques de 1 millón.

Seleccionamos la última manera. Necesitaremos guardar 25 vectores de 1 millón para poder calcular los percentiles de cada estilo. Dado que ese es el output.

Los datos del primer año, 2016, se usan únicamente como histórico

In [42]:
fechas_compra = pd.date_range('2017-01-01', df_fondos.index[-1], freq='BMS')
fechas_venta = pd.date_range('2017-01-01', df_fondos.index[-1], freq='BM')

### Inversión aleatoria

In [111]:
def monos_aleatorios(df_fondos, fechas_compra, fechas_venta, millones = 25):

    start_time = time.time()

    # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos aleatorios
    rent_acu_inver_ale = np.empty(0)

    for repeticion in range(millones):

        print(f'Generando millón {repeticion + 1} de inversiones aleatorias')

        rent_acu = np.zeros(1_000_000)

        for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

            # Extraemos el data set con el que vamos a trabajar
            universo_ini = df_fondos.loc[fecha_c,:]
            universo_fin = df_fondos.loc[fecha_v,:]

            # Calculo la inversión aleatoria para un bloque de 1 millón de monos
            id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)

            # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
            df = pd.concat([universo_ini, universo_fin], axis=1).T
            rent_periodo = np.log(df).diff().sum(axis=0)

            # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
            # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
            rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
            rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

            # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
            rent_monos = rent_fondos_mono * pesos_asignados
            rent_monos = rent_monos.sum(axis=1)

            # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
            rent_acu = rent_acu + rent_monos

            # Comprobamos cuanto ocupa en memoria rent_acu
            # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb

        rent_acu_inver_ale = np.append(rent_acu_inver_ale, rent_acu)
        # print(f'{sys.getsizeof(rent_acu_inver_ale)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

    # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
    cuantiles_aleatorios = np.quantile(rent_acu_inver_ale, np.linspace(0, 1, 21))

    tiempo = time.time() - start_time
    print(f'La generación de inversiones aleatorias ha tardado {tiempo/60/60} horas')
    
    return cuantiles_aleatorios

In [103]:
cuantiles_aleatorios = monos_aleatorios(df_fondos = df_fondos, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = 25)

Generando millón 1 de inversiones aleatorias
Generando millón 2 de inversiones aleatorias
Generando millón 3 de inversiones aleatorias
Generando millón 4 de inversiones aleatorias
Generando millón 5 de inversiones aleatorias
Generando millón 6 de inversiones aleatorias
Generando millón 7 de inversiones aleatorias
Generando millón 8 de inversiones aleatorias
Generando millón 9 de inversiones aleatorias
Generando millón 10 de inversiones aleatorias
Generando millón 11 de inversiones aleatorias
Generando millón 12 de inversiones aleatorias
Generando millón 13 de inversiones aleatorias
Generando millón 14 de inversiones aleatorias
Generando millón 15 de inversiones aleatorias
Generando millón 16 de inversiones aleatorias
Generando millón 17 de inversiones aleatorias
Generando millón 18 de inversiones aleatorias
Generando millón 19 de inversiones aleatorias
Generando millón 20 de inversiones aleatorias
Generando millón 21 de inversiones aleatorias
Generando millón 22 de inversiones aleatori

In [104]:
cuantiles_aleatorios

array([-1.86477612,  0.07547405,  0.10706573,  0.12381512,  0.13600911,
        0.14598731,  0.15471157,  0.16264212,  0.17008318,  0.17721309,
        0.18421067,  0.19119129,  0.19829481,  0.20566612,  0.21349062,
        0.22204416,  0.23176372,  0.24346186,  0.25923647,  0.28713208,
        1.33442439])

### Inversión en Alpha

In [246]:
def monos_alpha(df_fondos, msci, fechas_compra, fechas_venta, millones = 25, historico = 120):

    start_time = time.time()

    # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Alpha
    rent_acu_inver_alpha = np.empty(0)

    for repeticion in range(millones):

        print(f'Generando millón {repeticion + 1} de inversiones basadas en Alpha')

        rent_acu = np.zeros(1_000_000)

        for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

            # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
            historico_universo_alpha = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]
            ind_msci = msci[(fecha_c - timedelta(days=historico)):fecha_c]

            universo_elegible_alpha = universe_alpha(historico_universo_alpha, ind_msci)            
            
            # Comprobamos que hay suficientes activos que cumplen el criterio de alpha positivo (al menos 30). En caso contrario, se trabajará con el dataset completo.
            # Realmente aquí no hay que hacer nada, dado que el universo se adapta en la propia función de universe_alpha
            # print(f'El universo de activos que cumple la condición de alpha positivo es de {universo_elegible_alpha.sum()}')            
            df_fondos_alpha = df_fondos.loc[:,universo_elegible_alpha]            

            # Extraemos el data set con el que vamos a trabajar
            universo_ini = df_fondos_alpha.loc[fecha_c,:]
            universo_fin = df_fondos_alpha.loc[fecha_v,:]

            # Calculo la inversión aleatoria para un bloque de 1 millón de monos
            id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)    

            # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
            df = pd.concat([universo_ini, universo_fin], axis=1).T
            rent_periodo = np.log(df).diff().sum(axis=0)

            # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
            # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
            rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
            rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

            # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
            rent_monos = rent_fondos_mono * pesos_asignados
            rent_monos = rent_monos.sum(axis=1)

            # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
            rent_acu = rent_acu + rent_monos

            # Comprobamos cuanto ocupa en memoria rent_acu
            # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb
    
        rent_acu_inver_alpha = np.append(rent_acu_inver_alpha, rent_acu)
        # print(f'{sys.getsizeof(rent_acu_inver_alpha)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

    # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
    cuantiles_alpha = np.quantile(rent_acu_inver_alpha, np.linspace(0, 1, 21))

    tiempo = time.time() - start_time
    print(f'La generación de inversiones basadas en Alpha ha tardado {tiempo/60/60} horas')
    
    return cuantiles_alpha

In [247]:
cuantiles_alpha = monos_alpha(df_fondos = df_fondos, msci = msci, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = 25, historico = 120)

Generando millón 1 de inversiones basadas en Alpha
Generando millón 2 de inversiones basadas en Alpha
Generando millón 3 de inversiones basadas en Alpha
Generando millón 4 de inversiones basadas en Alpha
Generando millón 5 de inversiones basadas en Alpha
Generando millón 6 de inversiones basadas en Alpha
Generando millón 7 de inversiones basadas en Alpha
Generando millón 8 de inversiones basadas en Alpha
Generando millón 9 de inversiones basadas en Alpha
Generando millón 10 de inversiones basadas en Alpha
Generando millón 11 de inversiones basadas en Alpha
Generando millón 12 de inversiones basadas en Alpha
Generando millón 13 de inversiones basadas en Alpha
Generando millón 14 de inversiones basadas en Alpha
Generando millón 15 de inversiones basadas en Alpha
Generando millón 16 de inversiones basadas en Alpha
Generando millón 17 de inversiones basadas en Alpha
Generando millón 18 de inversiones basadas en Alpha
Generando millón 19 de inversiones basadas en Alpha
Generando millón 20 d

In [248]:
cuantiles_alpha

array([-1.57515249,  0.04631944,  0.08078318,  0.09785581,  0.11002121,
        0.11987004,  0.12838207,  0.13608325,  0.14328987,  0.15017783,
        0.15689285,  0.16357954,  0.170346  ,  0.17733844,  0.18473185,
        0.19277243,  0.20184235,  0.21270734,  0.22710054,  0.25179017,
        1.28219666])

### Inversión en Sharpe

In [41]:
def monos_sharpe(df_fondos, fechas_compra, fechas_venta, millones = 25, historico = 90, umbral = 0.5):

    start_time = time.time()

    # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Sharpe
    rent_acu_inver_sharpe = np.empty(0)

    for repeticion in range(millones):

        print(f'Generando millón {repeticion + 1} de inversiones basadas en Sharpe')

        rent_acu = np.zeros(1_000_000)

        for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

            # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
            historico_universo_sharpe = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]

            universo_elegible_sharpe = universe_sharpe(historico_universo_sharpe, df_fondos, umbral = umbral)                       
            
            # Comprobamos que hay suficientes activos que cumplen el criterio del umbral (al menos 30). En caso contrario, se trabajará con el dataset completo.
            # Realmente aquí no hay que hacer nada, dado que el universo se adapta en la propia función de universe_sharpe
            # print(f'El universo de activos que cumple la condición del umbral es de {universo_elegible_sharpe.sum()}')
            df_fondos_sharpe = df_fondos.loc[:,universo_elegible_sharpe]

            # Extraemos el data set con el que vamos a trabajar
            universo_ini = df_fondos_sharpe.loc[fecha_c,:]
            universo_fin = df_fondos_sharpe.loc[fecha_v,:]

            # Calculo la inversión aleatoria para un bloque de 1 millón de monos
            id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)    

            # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
            df = pd.concat([universo_ini, universo_fin], axis=1).T
            rent_periodo = np.log(df).diff().sum(axis=0)

            # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
            # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
            rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
            rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

            # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
            rent_monos = rent_fondos_mono * pesos_asignados
            rent_monos = rent_monos.sum(axis=1)

            # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
            rent_acu = rent_acu + rent_monos

            # Comprobamos cuanto ocupa en memoria rent_acu
            # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb
    
        rent_acu_inver_sharpe = np.append(rent_acu_inver_sharpe, rent_acu)
        # print(f'{sys.getsizeof(rent_acu_inver_sharpe)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

    # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
    cuantiles_sharpe = np.quantile(rent_acu_inver_sharpe, np.linspace(0, 1, 21))

    tiempo = time.time() - start_time
    print(f'La generación de inversiones basadas en Sharpe ha tardado {tiempo/60/60} horas')
    
    return cuantiles_sharpe

In [244]:
cuantiles_sharpe = monos_sharpe(df_fondos = df_fondos, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = 25, historico = 90, umbral = 0.0)

Generando millón 1 de inversiones basadas en Sharpe
Generando millón 2 de inversiones basadas en Sharpe
Generando millón 3 de inversiones basadas en Sharpe
Generando millón 4 de inversiones basadas en Sharpe
Generando millón 5 de inversiones basadas en Sharpe
Generando millón 6 de inversiones basadas en Sharpe
Generando millón 7 de inversiones basadas en Sharpe
Generando millón 8 de inversiones basadas en Sharpe
Generando millón 9 de inversiones basadas en Sharpe
Generando millón 10 de inversiones basadas en Sharpe
Generando millón 11 de inversiones basadas en Sharpe
Generando millón 12 de inversiones basadas en Sharpe
Generando millón 13 de inversiones basadas en Sharpe
Generando millón 14 de inversiones basadas en Sharpe
Generando millón 15 de inversiones basadas en Sharpe
Generando millón 16 de inversiones basadas en Sharpe
Generando millón 17 de inversiones basadas en Sharpe
Generando millón 18 de inversiones basadas en Sharpe
Generando millón 19 de inversiones basadas en Sharpe
Ge

In [245]:
cuantiles_sharpe

array([-1.78240996, -0.02879209,  0.00427916,  0.02071316,  0.03244229,
        0.04195612,  0.05021449,  0.0576909 ,  0.06467702,  0.07136937,
        0.07788885,  0.08437506,  0.09093926,  0.0977313 ,  0.1048976 ,
        0.11266138,  0.12137796,  0.13170867,  0.14517087,  0.16699387,
        1.03790409])

### Inversión en Markowitz

In [241]:
def monos_markowitz(df_fondos, fechas_compra, fechas_venta, millones = 25, historico = 365, num_fondos_por_cartera = 100, num_carteras = 250_000, p = 0.85, porcentaje_util = 0.2):

    start_time = time.time()

    # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Markowitz
    rent_acu_inver_mark = np.empty(0)

    for repeticion in range(int((millones*1_000_000) /(num_carteras*porcentaje_util))):

        print(f'Generando inversión {(repeticion + 1)*50_000} basadas en Markowitz')   

        rent_acu = np.zeros(int(num_carteras*porcentaje_util))

        for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

            # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
            historico_universo_mark = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]     

            # Extraemos el data set con el que vamos a trabajar
            universo_ini = df_fondos.loc[fecha_c,:]
            universo_fin = df_fondos.loc[fecha_v,:]

            # Calculo la inversión basada en la frontera de Markowitz para un bloque de N monos    
            id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = frontera_markowitz_fast(historico_universo_mark, 
                                                                                                      num_fondos_por_cartera = num_fondos_por_cartera, 
                                                                                                      num_carteras = num_carteras,
                                                                                                      p = p, 
                                                                                                      porcentaje_util = porcentaje_util)

            # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
            df = pd.concat([universo_ini, universo_fin], axis=1).T
            rent_periodo = np.log(df).diff().sum(axis=0)

            # En el caso de Markowitz, todas las inversiones trabajan con los mismos 100 activos. Lo que difiere es el peso que se le da a cada uno.
            rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos]

            # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
            rent_monos = rent_fondos_mono.values * pesos_asignados
            rent_monos = rent_monos.sum(axis=1)

            # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
            rent_acu = rent_acu + rent_monos

            # Comprobamos cuanto ocupa en memoria rent_acu
            # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 0,4 mb
    
        rent_acu_inver_mark = np.append(rent_acu_inver_mark, rent_acu)
        # print(f'{sys.getsizeof(rent_acu_inver_sharpe)/1_000_000} mb ocupa id_fondos_elegidos')

    # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
    cuantiles_mark = np.quantile(rent_acu_inver_mark, np.linspace(0, 1, 21))

    tiempo = time.time() - start_time
    print(f'La generación de inversiones basadas en Markowitz ha tardado {tiempo/60/60} horas')
    
    return cuantiles_mark        

In [242]:
cuantiles_markowitz = monos_markowitz(df_fondos, fechas_compra, fechas_venta, millones = 25, historico = 365, num_fondos_por_cartera = 100, num_carteras = 250_000, p = 0.85, porcentaje_util = 0.2)

Generando inversión 50000 basadas en Markowitz
Generando inversión 100000 basadas en Markowitz
Generando inversión 150000 basadas en Markowitz
Generando inversión 200000 basadas en Markowitz
Generando inversión 250000 basadas en Markowitz
Generando inversión 300000 basadas en Markowitz
Generando inversión 350000 basadas en Markowitz
Generando inversión 400000 basadas en Markowitz
Generando inversión 450000 basadas en Markowitz
Generando inversión 500000 basadas en Markowitz
Generando inversión 550000 basadas en Markowitz
Generando inversión 600000 basadas en Markowitz
Generando inversión 650000 basadas en Markowitz
Generando inversión 700000 basadas en Markowitz
Generando inversión 750000 basadas en Markowitz
Generando inversión 800000 basadas en Markowitz
Generando inversión 850000 basadas en Markowitz
Generando inversión 900000 basadas en Markowitz
Generando inversión 950000 basadas en Markowitz
Generando inversión 1000000 basadas en Markowitz
Generando inversión 1050000 basadas en M

### Ponemos el resultado final en bonito

In [249]:
dic_cuantiles = {'Aleatorio': cuantiles_aleatorios, 'Alpha': cuantiles_alpha, 'Sharpe': cuantiles_sharpe, 'Markowitz': cuantiles_markowitz}
indice = [f"{round(i*100,0)}%" for i in np.linspace(0, 1, 21)]

df_result = pd.DataFrame(dic_cuantiles, index = indice)
df_result['Global'] = df_result.mean(axis=1)
df_result

,Aleatorio,Alpha,Sharpe,Markowitz,Global
0.0%,-1.864776,-1.575152,-1.782410,-3.162409,-2.096187
5.0%,0.075474,0.046319,-0.028792,0.083259,0.044065
10.0%,0.107066,0.080783,0.004279,0.115392,0.076880
15.0%,0.123815,0.097856,0.020713,0.133423,0.093952
20.0%,0.136009,0.110021,0.032442,0.146823,0.106324
25.0%,0.145987,0.119870,0.041956,0.157910,0.116431
30.0%,0.154712,0.128382,0.050214,0.167667,0.125244
35.0%,0.162642,0.136083,0.057691,0.176583,0.133250
40.0%,0.170083,0.143290,0.064677,0.184991,0.140760
45.0%,0.177213,0.150178,0.071369,0.193068,0.147957


# Lo ponemos todo en una clase y lo instanciamos

In [39]:
class monos:
    
    def __init__(self, freq='1BMS', mill=100, ale=0.25, alpha=0.25, sharpe=0.25, mark=0.25, hist_alpha=120, hist_sharpe=90, umbral_sharpe=0.0, hist_mark=365, nfondos_mark = 100, 
                 ncarteras_mark = 250_000, p_mark = 0.85, porc_mark = 0.2):
        
        self.freq = freq
        self.mill = mill
        self.ale = ale
        self.alpha = alpha
        self.sharpe = sharpe
        self.mark = mark
        self.hist_alpha = hist_alpha
        self.hist_sharpe= hist_sharpe
        self.umbral_sharpe = umbral_sharpe
        self.hist_mark = hist_mark
        self.nfondos_mark = nfondos_mark
        self.ncarteras_mark = ncarteras_mark
        self.p_mark = p_mark
        self.porc_mark = porc_mark

    def generar (self):    
        
        start_time = time.time()
        
        try:
            # Leemos los datos que necesitamos para hacer el ejercicio
            df_fondos, msci = self.carga_limpieza()
            
        except:            
            print("No ha sido posible cargar los datos")  
            return()
        
        # Establecemos los periodos de compra y venta
        fechas_compra = pd.date_range('2017-01-01', df_fondos.index[-1], freq=self.freq)
        fechas_venta = pd.date_range('2017-01-01', df_fondos.index[-1], freq=self.freq[0:len(self.freq)-1])
        
        # Realizamos los distintos tipo de inversiones
        cuantiles_aleatorios = self.monos_aleatorios(df_fondos = df_fondos, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = int(self.mill * self.ale))
        
        cuantiles_alpha = self.monos_alpha(df_fondos = df_fondos, msci = msci, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = int(self.mill * self.alpha), historico = self.hist_alpha)
        
        cuantiles_sharpe = self.monos_sharpe(df_fondos = df_fondos, fechas_compra = fechas_compra, fechas_venta = fechas_venta, millones = int(self.mill * self.sharpe), 
                                             historico = self.hist_sharpe, umbral = self.umbral_sharpe)
        
        cuantiles_markowitz = self.monos_markowitz(df_fondos, fechas_compra, fechas_venta, millones = int(self.mill * self.mark), 
                                                   historico = self.hist_mark, num_fondos_por_cartera = self.nfondos_mark, num_carteras = self.ncarteras_mark, 
                                                   p = self.p_mark, porcentaje_util = self.porc_mark)
        
        # Calculamos el DF resultado
        dic_cuantiles = {'Aleatorio': cuantiles_aleatorios, 'Alpha': cuantiles_alpha, 'Sharpe': cuantiles_sharpe, 'Markowitz': cuantiles_markowitz}
        indice = [f"{round(i*100,0)}%" for i in np.linspace(0, 1, 21)]

        df_result = pd.DataFrame(dic_cuantiles, index = indice)
        df_result['Global'] = df_result.mean(axis=1)
            
        tiempo = time.time() - start_time
        print(f'La generación de {self.mill} millones de monos ha tardado {tiempo/60/60} horas') 
        
        return(df_result)
       
    def carga_limpieza (self): 
        
        print('Iniciando proceso de carga y limpieza de datos')
        
        navs = pd.read_pickle('navs.pickle')
        keys = list(navs.keys())
        
        # Homogeneizamos       
        fecha_inicial = navs[keys[0]].index[0]
        fecha_final = navs[keys[0]].index[-1]

        for key in navs.keys():

            if navs[key].index[0] < fecha_inicial:

                fecha_inicial = navs[key].index[0]

            if navs[key].index[-1] > fecha_final:

                fecha_final = navs[key].index[-1]
        
        # Cargamos el MSCI
        msci= pd.read_csv("MSCI.csv", index_col="Fecha",
                  usecols=["Fecha", "Último"],
                  parse_dates=True,
                  infer_datetime_format= "%Y-%m-%d",
                  thousands=".", decimal=",")

        msci.rename(columns={'Último': 'MSCI'}, inplace=True)
        msci.sort_index(inplace=True)
        
        # Cargamos el maestro de valores
        maestro = pd.read_csv('maestro.csv')
        
        # Extraemos los identificadores del maestro de valores (allfunds_id)
        identificadores = maestro.loc[:,'allfunds_id']

        # El número de fondos no coincide en el pickle de navs y el maestro. Buscamos las diferencias
        list_difference = [item for item in identificadores if item not in keys]
        
        # Generamos un DF con el que trabajar
        # Ponemos los fondos que tenemos en (columnas) y fechas (filas)
        fechas = pd.date_range(start=fecha_inicial,end=fecha_final, freq='B')
        df_fondos = pd.DataFrame(np.zeros((fechas.shape[0], identificadores.shape[0])),index=fechas,columns=identificadores)

        for funds_id in navs.keys():
            df_fondos.loc[:,funds_id] = navs[funds_id].nav
        
        # Nos quedamos con aquellos fondos que tengan más del 60% de los datos rellenos
        df_fondos = df_fondos.loc[:,df_fondos.isna().sum(axis=0)<df_fondos.shape[0]*0.4]
        
        # Rellenamos los datos faltantes
        df_fondos = df_fondos.fillna(method = 'ffill')
        df_fondos = df_fondos.fillna(method = 'bfill')

        # Quitamos aquellos fondos que tienen nav 0 en todas sus fechas: los fondos que estaban en el maestro de valores, pero que no tenían Nav
        df_fondos=df_fondos.drop(list_difference, axis=1)

        return df_fondos, msci

    def universe_alpha(self, universe, ind_msci):

        '''
        Alpha de Jensen: invertirá en N fondos (variable) que tengan Alpha positiva en relación con el MSCI. 
        La ventana de datos con la que hacer el cálculo será de 120 días para cada mono.

        La función del Alpha calculará el universo de fondos que cumplen la condición de Alpha positivo en relación al MSCI, en un momento dado. 
        Una vez tengamos el universo, la inversión será aleatoria en dicho universo.
        En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

        La función recibe los DF de fondos y msci, y devuelve un índice booleano que modifica el universo de inverión aleatoria.
        α=Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))
        β= covarianza (Rentabilidad_cartera, Rentabilidad_mercado) / varianza (Rentabilidad_mercado)
        '''

        # Generamos las rentabilidades logarítmicas para el índice, la rentabilidad libre de riesgo y los activos.
        rent_activos = np.log(universe).diff().dropna()
        rent_msci = np.log(ind_msci).diff().dropna()
        rent_libre_riesgo = 0

        # Calculamos la varianza
        varianza_msci = rent_msci.var()

        # Calculamos la covarianza
        cov_act_msci = pd.concat([rent_activos,rent_msci], axis=1).cov()
        cov_act_msci = cov_act_msci.iloc[0:-1,-1] 

        # Calculamos la Beta y el Alpha
        if varianza_msci[0] == 0:

            # beta = 1
            alpha = rent_activos.iloc[-1,:] - rent_msci.iloc[-1,0]

        else:

            beta = cov_act_msci/varianza_msci[0]

            # Calculamos el Alpha para cada activo: Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))
            alpha = rent_activos.iloc[-1,:] - beta.iloc[:] * rent_msci.iloc[-1,0]

        # Devolvemos el universo de activos elegibles: aquellos que tienen alpha > 0
        # Si ningún activo cumpliera el criterio, el universo de activos elegibles sería el inicial (todos los activos).
        if sum(alpha > 0)==0:
            universo_elegible_alpha = alpha < 0
        else:
            universo_elegible_alpha = alpha > 0

        return universo_elegible_alpha
    
    def universe_sharpe(self, universe, df_fondos, umbral=0.5):

        '''
        Sharpe: invertirá en N fondos (variable), que tengan un ratio de Sharpe mayor a un umbral de 0,5. 
        La ventana de datos con la que hacer el cálculo será de 90 días para cada mono.

        La función calculará el universo de fondos que cumplen la condición de Sharpe > umbral, en un momento dado. 
        Una vez tengamos el universo, la inversión será aleatoria en dicho universo.
        En caso de que el universo esté vacío, se realizará una inversión aleatoria sobre el universo completo.

        La función recibe los DF de fondos y un umbral, y devuelve un índice booleano que modifica el universo de inverión aleatoria.
        Ratio de Sharpe= (Rent activo- Rentabilidad libre de riesgo)/ desviación típica activo
        '''

        # Generamos las rentabilidades para los activos y calculamos la desviación típica
        rent_activos = np.log(universe).diff().dropna()
        rent_libre_riesgo = 0
        desv_act = df_fondos.std(axis=0)

        # Calculamos el ratio de Sharpe
        sharpe = rent_activos.iloc[-1,:]/desv_act

        # Las desviaciones que eran = 0 se han convertido en NA. Lo solventamos
        sharpe[desv_act == 0] = 0

        # Devolvemos el universo de activos elegibles: aquellos que tienen Sharpe > 0.5
        # Si ningún activo cumpliera el criterio, el universo de activos elegibles sería el inicial (todos los activos).
        if sum(sharpe > umbral)==0:
            universo_elegible_sharpe = sharpe < umbral
        else:
            universo_elegible_sharpe = sharpe > umbral

        return universo_elegible_sharpe    
    
    def inversion_aleatoria_fast(self, universe, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25):

        '''
        Método de inversión aleatoria capaz de obtener gran cantidad de carteras, de tamaño aleatorio, de una sola vez
        '''

        universe_size = universe.size

        posiciones_fondos_elegidos = np.random.choice(universe_size, num_fondos_por_cartera * num_carteras, replace=True)
        id_fondos_elegidos = universe.index.values[posiciones_fondos_elegidos]
        pesos_asignados = np.random.random(num_fondos_por_cartera * num_carteras)

        # f'{sys.getsizeof(posiciones_fondos_elegidos)/1_000_000} mb' # Nos da el tamaño en RAM que ocupa el objeto
        posiciones_fondos_elegidos.resize(num_carteras,num_fondos_por_cartera) # Mejor que reshape porque resize lo hace inplace
        id_fondos_elegidos.resize(num_carteras,num_fondos_por_cartera)
        pesos_asignados.resize(num_carteras,num_fondos_por_cartera)

        # Creamos una matriz booleana para anular aleatoriamente los pesos de N fondos de cada cartera (fila)
        anular = np.random.choice(a=[False, True], size=(num_carteras, num_fondos_por_cartera), p=[p, 1-p])  
        pesos_asignados = pesos_asignados * anular

        # Hay que reescalar los pesos por filas (por carteras)
        pesos_asignados = pesos_asignados/pesos_asignados.sum(axis=1, keepdims=True)

        return id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos
    
    def frontera_markowitz_fast(self, universe, num_fondos_por_cartera = 100, num_carteras = 500_000, p = 0.85, porcentaje_util = 0.2):
        '''
        El objetivo es que la función arroje 100.000 carteras (num_carteras * porcentaje_util), si bien no óptimas, sí eficientes.
        El número óptimo de fondos con los que la función trabaja es 100, por tiempo de ejecución y tamaño en RAM
        Eso no significa que cada cartera tenga 100 activos. Cada cartera tendrá un número distinto de activos, gracias al proceso de anulación.
        Se genera una matriz booleana, con una probabilidad de False de p. El objetivo es que las carteras tengan entre 5 y 30 activos.
        De las 500.000 carteras calculadas, la función se queda con el percentil 80, para arriba. 
        '''

        # Sacamos la matriz de pesos para cada cartera
        pesos_asignados = np.random.random(num_fondos_por_cartera * num_carteras)
        pesos_asignados.resize(num_carteras,num_fondos_por_cartera)

        # Creamos una matriz booleana para anular aleatoriamente los pesos de N fondos de cada cartera (fila)
        anular = np.random.choice(a=[False, True], size=(num_carteras, num_fondos_por_cartera), p=[p, 1-p])  
        pesos_asignados = pesos_asignados * anular      

        # Quitamos aquellas carteras que no tengan ningún activo con peso asignado
        pesos_asignados = pesos_asignados[anular.sum(axis=1) != 0,:]    

        # Hay que reescalar los pesos por filas (por carteras)
        matriz_pesos = pesos_asignados/pesos_asignados.sum(axis=1, keepdims=True)

        # Elegimos aleatoriamente los activos con los que trabajar (sub universo)    
        universe_size = universe.shape[1]
        posiciones_fondos_elegidos = np.random.choice(universe_size, num_fondos_por_cartera, replace=False)
        id_fondos_elegidos = universe.columns[posiciones_fondos_elegidos]
        sub_universo = universe.iloc[:,posiciones_fondos_elegidos]

        # Calculamos la rentabilidad de los activos
        rent_activos = np.log(sub_universo).diff().dropna()

        # Calculamos la matriz de varianzas / covarianzas
        matriz_covarianzas = rent_activos.cov()
        matriz_covarianzas = matriz_covarianzas.to_numpy(dtype='float') 

        # Calculamos la rentabilidad diaria del periodo para cada activo: LN(precio final/ precio inicial)/nº de datos
        precio_inicial = sub_universo.iloc[0,:]
        precio_final = sub_universo.iloc[-1,:]
        rentabilidad_diaria = np.log(precio_final / precio_inicial) / sub_universo.shape[0]

        # Calculamos la rentabilidad de la cartera en función de los pesos 
        auxiliar = rentabilidad_diaria.values * matriz_pesos
        rentabilidad_carteras = auxiliar.sum(axis=1)

        # Calculamos el riesgo de la cartera (desviación), en función de los pesos
        auxiliar = np.dot(matriz_pesos, matriz_covarianzas) # Reutilizo la matriz auxiliar con otro propósito para no sobrecargar la RAM
        riesgo_carteras = pow((auxiliar * matriz_pesos).sum(axis=1), 0.5)

        # Calculamos la eficiencia de la cartera (pendiente), en función de los pesos
        # Existen posiciones que van a dar error, 0/0. Las localizo y hago que 0/1 = 0
        posiciones_0entre0 = (rentabilidad_carteras==0) * (riesgo_carteras==0)
        riesgo_carteras[posiciones_0entre0] = 0.0000001
        eficiencia_carteras = rentabilidad_carteras / riesgo_carteras

        # Localizamos las carteras con mayor eficiencia
        umbral_eficiencia = np.quantile(eficiencia_carteras, (1-porcentaje_util))    
        pesos_asignados = matriz_pesos[eficiencia_carteras>umbral_eficiencia,:]

        return id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos
    
    
    def monos_aleatorios(self, df_fondos, fechas_compra, fechas_venta, millones = 25):

        # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos aleatorios
        rent_acu_inver_ale = np.empty(0)

        for repeticion in range(millones):

            print(f'Generando millón {repeticion + 1} de inversiones aleatorias')

            rent_acu = np.zeros(1_000_000)

            for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

                # Extraemos el data set con el que vamos a trabajar
                universo_ini = df_fondos.loc[fecha_c,:]
                universo_fin = df_fondos.loc[fecha_v,:]

                # Calculo la inversión aleatoria para un bloque de 1 millón de monos
                id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = self.inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)

                # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
                df = pd.concat([universo_ini, universo_fin], axis=1).T
                rent_periodo = np.log(df).diff().sum(axis=0)

                # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
                # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
                rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
                rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

                # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
                rent_monos = rent_fondos_mono * pesos_asignados
                rent_monos = rent_monos.sum(axis=1)

                # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
                rent_acu = rent_acu + rent_monos

                # Comprobamos cuanto ocupa en memoria rent_acu
                # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb

            rent_acu_inver_ale = np.append(rent_acu_inver_ale, rent_acu)
            # print(f'{sys.getsizeof(rent_acu_inver_ale)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

        # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
        cuantiles_aleatorios = np.quantile(rent_acu_inver_ale, np.linspace(0, 1, 21))

        return cuantiles_aleatorios
    
    
    def monos_alpha(self, df_fondos, msci, fechas_compra, fechas_venta, millones = 25, historico = 120):

        # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Alpha
        rent_acu_inver_alpha = np.empty(0)

        for repeticion in range(millones):

            print(f'Generando millón {repeticion + 1} de inversiones basadas en Alpha')

            rent_acu = np.zeros(1_000_000)

            for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

                # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
                historico_universo_alpha = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]
                ind_msci = msci[(fecha_c - timedelta(days=historico)):fecha_c]

                universo_elegible_alpha = self.universe_alpha(historico_universo_alpha, ind_msci)            

                # Comprobamos que hay suficientes activos que cumplen el criterio de alpha positivo (al menos 30). En caso contrario, se trabajará con el dataset completo.
                # Realmente aquí no hay que hacer nada, dado que el universo se adapta en la propia función de universe_alpha
                # print(f'El universo de activos que cumple la condición de alpha positivo es de {universo_elegible_alpha.sum()}')            
                df_fondos_alpha = df_fondos.loc[:,universo_elegible_alpha]            

                # Extraemos el data set con el que vamos a trabajar
                universo_ini = df_fondos_alpha.loc[fecha_c,:]
                universo_fin = df_fondos_alpha.loc[fecha_v,:]

                # Calculo la inversión aleatoria para un bloque de 1 millón de monos
                id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = self.inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)    

                # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
                df = pd.concat([universo_ini, universo_fin], axis=1).T
                rent_periodo = np.log(df).diff().sum(axis=0)

                # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
                # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
                rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
                rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

                # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
                rent_monos = rent_fondos_mono * pesos_asignados
                rent_monos = rent_monos.sum(axis=1)

                # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
                rent_acu = rent_acu + rent_monos

                # Comprobamos cuanto ocupa en memoria rent_acu
                # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb

            rent_acu_inver_alpha = np.append(rent_acu_inver_alpha, rent_acu)
            # print(f'{sys.getsizeof(rent_acu_inver_alpha)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

        # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
        cuantiles_alpha = np.quantile(rent_acu_inver_alpha, np.linspace(0, 1, 21))

        return cuantiles_alpha
    
    
    def monos_sharpe(self, df_fondos, fechas_compra, fechas_venta, millones = 25, historico = 90, umbral = 0.5):

        # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Sharpe
        rent_acu_inver_sharpe = np.empty(0)

        for repeticion in range(millones):

            print(f'Generando millón {repeticion + 1} de inversiones basadas en Sharpe')

            rent_acu = np.zeros(1_000_000)

            for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

                # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
                historico_universo_sharpe = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]

                universo_elegible_sharpe = self.universe_sharpe(historico_universo_sharpe, df_fondos, umbral = self.umbral_sharpe)                       

                # Comprobamos que hay suficientes activos que cumplen el criterio del umbral (al menos 30). En caso contrario, se trabajará con el dataset completo.
                # Realmente aquí no hay que hacer nada, dado que el universo se adapta en la propia función de universe_sharpe
                # print(f'El universo de activos que cumple la condición del umbral es de {universo_elegible_sharpe.sum()}')
                df_fondos_sharpe = df_fondos.loc[:,universo_elegible_sharpe]

                # Extraemos el data set con el que vamos a trabajar
                universo_ini = df_fondos_sharpe.loc[fecha_c,:]
                universo_fin = df_fondos_sharpe.loc[fecha_v,:]

                # Calculo la inversión aleatoria para un bloque de 1 millón de monos
                id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = self.inversion_aleatoria_fast(universo_ini, num_fondos_por_cartera = 30, num_carteras = 1_000_000, p = 0.25)    

                # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
                df = pd.concat([universo_ini, universo_fin], axis=1).T
                rent_periodo = np.log(df).diff().sum(axis=0)

                # Cada mono tiene 30 fondos distintos. Creo una matriz con la rentabilidad de cada fondo, para cada mono: 1.000.000 * 30
                # Para poder crear la matriz, debo indexsar las rentabilidades de cada fondo, de cada mono. Para poder hacerlo, debo pasar una serie de una sola dimensión a rent_periodo
                rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos.flatten()]
                rent_fondos_mono = rent_fondos_mono.values.reshape(posiciones_fondos_elegidos.shape[0],posiciones_fondos_elegidos.shape[1]) # Para hacer reshape de una serie debo usar sus valores.   

                # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
                rent_monos = rent_fondos_mono * pesos_asignados
                rent_monos = rent_monos.sum(axis=1)

                # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
                rent_acu = rent_acu + rent_monos

                # Comprobamos cuanto ocupa en memoria rent_acu
                # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 8 mb

            rent_acu_inver_sharpe = np.append(rent_acu_inver_sharpe, rent_acu)
            # print(f'{sys.getsizeof(rent_acu_inver_sharpe)/1_000_000} mb ocupa id_fondos_elegidos') # 200 mb

        # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
        cuantiles_sharpe = np.quantile(rent_acu_inver_sharpe, np.linspace(0, 1, 21))

        return cuantiles_sharpe
    
    
    def monos_markowitz(self, df_fondos, fechas_compra, fechas_venta, millones = 25, historico = 365, num_fondos_por_cartera = 100, num_carteras = 250_000, p = 0.85, porcentaje_util = 0.2):

        # Generamos un array vacío, donde guardaremos los resultados de las inversiones de todos los monos basados en Markowitz
        rent_acu_inver_mark = np.empty(0)

        for repeticion in range(int((millones*1_000_000) /(num_carteras*porcentaje_util))):

            print(f'Generando inversión {(repeticion + 1)*50_000} basadas en Markowitz')   

            rent_acu = np.zeros(int(num_carteras*porcentaje_util))

            for fecha_c, fecha_v in zip(fechas_compra, fechas_venta):

                # Para calcular el universo elegible en función del Alpha es necesario un histórico de 120 días de df_fondos. Y lo mismo para el MSCI
                historico_universo_mark = df_fondos[(fecha_c - timedelta(days=historico)):fecha_c]     

                # Extraemos el data set con el que vamos a trabajar
                universo_ini = df_fondos.loc[fecha_c,:]
                universo_fin = df_fondos.loc[fecha_v,:]

                # Calculo la inversión basada en la frontera de Markowitz para un bloque de N monos    
                id_fondos_elegidos, pesos_asignados, posiciones_fondos_elegidos = self.frontera_markowitz_fast(historico_universo_mark, 
                                                                                                               num_fondos_por_cartera = num_fondos_por_cartera, 
                                                                                                               num_carteras = num_carteras,
                                                                                                               p = p, 
                                                                                                               porcentaje_util = porcentaje_util)

                # Calculo la rentabilidad del periodo para todos los fondos (desde fecha de compra, hasta fecha de venta)
                df = pd.concat([universo_ini, universo_fin], axis=1).T
                rent_periodo = np.log(df).diff().sum(axis=0)

                # En el caso de Markowitz, todas las inversiones trabajan con los mismos 100 activos. Lo que difiere es el peso que se le da a cada uno.
                rent_fondos_mono = rent_periodo.iloc[posiciones_fondos_elegidos]

                # Multiplico la matriz de rentabilidades por la matriz de pesos y sumo por filas, para obtener el vector de rentabilidades de cada mono
                rent_monos = rent_fondos_mono.values * pesos_asignados
                rent_monos = rent_monos.sum(axis=1)

                # Calculo la rentabilidad acumulada (al ser logarítmicas, las puedo sumar)
                rent_acu = rent_acu + rent_monos

                # Comprobamos cuanto ocupa en memoria rent_acu
                # print(f'{sys.getsizeof(rent_acu)/1_000_000} mb ocupa id_fondos_elegidos') # 0,4 mb

            rent_acu_inver_mark = np.append(rent_acu_inver_mark, rent_acu)
            # print(f'{sys.getsizeof(rent_acu_inver_sharpe)/1_000_000} mb ocupa id_fondos_elegidos')

        # Calculamos los cuantiles de 0% a 100%, de 5% en 5%
        cuantiles_mark = np.quantile(rent_acu_inver_mark, np.linspace(0, 1, 21))

        return cuantiles_mark  

In [40]:
mono_guerra_paz = monos(freq='1BMS', mill=100, ale=0.25, alpha=0.25, sharpe=0.25, mark=0.25, hist_alpha=120, hist_sharpe=90, umbral_sharpe=0.0, hist_mark=365, 
                        nfondos_mark = 100, ncarteras_mark = 250_000, p_mark = 0.85, porc_mark = 0.2)

resultado = mono_guerra_paz.generar()

Iniciando proceso de carga y limpieza de datos
Generando millón 1 de inversiones aleatorias
Generando millón 2 de inversiones aleatorias
Generando millón 3 de inversiones aleatorias
Generando millón 4 de inversiones aleatorias
Generando millón 5 de inversiones aleatorias
Generando millón 6 de inversiones aleatorias
Generando millón 7 de inversiones aleatorias
Generando millón 8 de inversiones aleatorias
Generando millón 9 de inversiones aleatorias
Generando millón 10 de inversiones aleatorias
Generando millón 11 de inversiones aleatorias
Generando millón 12 de inversiones aleatorias
Generando millón 13 de inversiones aleatorias
Generando millón 14 de inversiones aleatorias
Generando millón 15 de inversiones aleatorias
Generando millón 16 de inversiones aleatorias
Generando millón 17 de inversiones aleatorias
Generando millón 18 de inversiones aleatorias
Generando millón 19 de inversiones aleatorias
Generando millón 20 de inversiones aleatorias
Generando millón 21 de inversiones aleator

In [43]:
resultado

,Aleatorio,Alpha,Sharpe,Markowitz,Global
0.0%,-1.433748,-1.312177,-1.795629,-2.881668,-1.855805
5.0%,0.075471,0.046288,-0.028890,0.084418,0.044321
10.0%,0.107071,0.080797,0.004272,0.115242,0.076846
15.0%,0.123821,0.097856,0.020699,0.132958,0.093833
20.0%,0.135997,0.109999,0.032426,0.146222,0.106161
25.0%,0.145987,0.119840,0.041930,0.157250,0.116252
30.0%,0.154706,0.128368,0.050200,0.166960,0.125059
35.0%,0.162646,0.136072,0.057687,0.175858,0.133066
40.0%,0.170080,0.143276,0.064670,0.184237,0.140566
45.0%,0.177224,0.150153,0.071354,0.192292,0.147756
